In [19]:
from backtesting import Backtest, Strategy
import pandas as pd
import ta

In [20]:
df = pd.read_csv("BNB_df.csv")
macd = pd.read_csv("BNB_macd.csv")
signal = pd.read_csv("Singanls.csv")

In [10]:
class Macd():




IndentationError: expected an indented block after class definition on line 1 (1658956194.py, line 2)